# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [1]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}
for key,value in obj.items():
    if isinstance(value, str):
        print(f'{key:<10} = {value}')
    else:
        print(f'{key:<10} = {value}')

home_page  = https://github.com/pypa/sampleproject
keywords   = sample setuptools development
license    = MIT


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [59]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

# выбираем случайные 5 рецептов и нужные столбцы
random_recipes = recipes.sample(5)[['id', 'minutes']]

# форматируем значения столбца id с выводом в таблицу
table_str = random_recipes.apply(lambda row: "|{:^{width}}|{:^{width}}|\n".format(row.id, row.minutes, width=max(len(str(max(random_recipes.id))), len("id")) + 2),axis=1)

# заголовок таблицы
header = "|{:^{width}}|{:^{width}}|\n".format("id", "minutes", width=max(len(str(max(random_recipes.id))), len("id"))+2)
  
print(header + "-"*len(header) + table_str.to_string(index=False, header=False))

|   id   |minutes |
--------------------| 212342 |   25   |\n
| 395623 |   18   |\n
| 421620 |   35   |\n
| 86668  |   45   |\n
| 279981 |  245   |\n


In [67]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv')

sample_recipes = recipes.sample(n=5)[['id', 'minutes']]

for col in sample_recipes.columns:
    max_width = max(sample_recipes[col].apply(lambda x: len(str(x))))
    print(f"|{col:<{max_width + 2}}", end="")
print("|")
print("|" + "-" * (max_width + 14) + "|")

for idx, row in sample_recipes.iterrows():
    for col in sample_recipes.columns:
        max_width = max(sample_recipes[col].apply(lambda x: len(str(x))))
        print(f"|{row[col]:<{max_width + 3}}", end="")
    print("|")

|id      |minutes|
|-----------------|
|162572   |505   |
|439595   |35    |
|92548    |75    |
|341126   |15    |
|85943    |75    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

<IPython.core.display.Javascript object>

In [31]:
import csv

def read_recipe_data(recipe_id):
    with open('recipes_sample.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['id'] == recipe_id:
                return {
                    'name': row['name'],
                    'contributor_id': row['contributor_id'],
                    'minutes': int(row['minutes'])
                }
import xml.etree.ElementTree as ET

def read_steps_data(recipe_id):
    tree = ET.parse('steps_sample.xml')
    root = tree.getroot()
    
    steps = []
    for step in root.findall(f".//recipe[@id='{recipe_id}']/recipetext/*"):
        if step.tag == 'step':
            steps.append(step.text.strip())
            
    return steps
def show_info(recipe_id):
    recipe_data = read_recipe_data(recipe_id)
    steps_data = read_steps_data(recipe_id)
    
    name = recipe_data['name'].title()
    
    steps_str = '\n'.join([f"{i}. {name}" for i, step in enumerate(steps_data, start=1)])
    
    contributor_id = recipe_data['contributor_id']
    
    minutes = recipe_data['minutes']
    info_str = f"Название {name}\n{steps_str}\n{'-'*10}\nАвтор: {contributor_id}\nСреднее время приготовления: {minutes} минут"
    
    return info_str
print(show_info('170895'))

Название Leeks And Parsnips  Sauteed Or Creamed

----------
Автор: 8377
Среднее время приготовления: 27 минут


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [12]:
import pandas as pd
import re


data = pd.read_excel('steps_sample.xlsx')


pattern = re.compile(r'\d+\s(hour|hours|minute|minutes)')


for i in range(len(data)):
    result = re.search(pattern, str(data.loc[i, 'step']))
    if result:
        print(result.group())

5 minute
5 minute
5 minute
5 minute
15 minute
15 minute
25 minute
45 minute
20 minute
14 minute
20 minute
10 minute
50 minute
15 minute
3 hour
40 minute
30 minute
20 minute
45 minute
10 minute
1 minute
40 minute
4 hour
30 minute
45 minute
10 minute
15 minute
2 hour
15 minute
15 minute
45 minute
15 minute
10 minute
10 minute
1 minute
3 minute
50 minute
7 minute
30 minute
40 minute
7 minute
20 minute
10 minute
1 minute
5 minute
10 minute
5 minute
25 minute
15 minute
2 minute
1 minute
45 minute
2 minute
1 minute
24 minute
7 hour
1 hour
1 hour
18 minute
65 minute
2 minute
30 minute
3 minute
2 minute
5 minute
3 minute
4 minute
3 minute
60 minute
15 minute
15 minute
35 minute
10 minute
5 minute
1 hour
10 minute
15 minute
40 minute
25 minute
15 minute
5 minute
45 minute
45 minute
30 minute
1 hour
60 minute
8 hour
2 minute
3 minute
1 hour
60 minute
2 minute
3 minute
50 minute
40 minute
10 minute
1 minute
5 minute
25 minute
30 minute
55 minute
3 minute
10 minute
2 hour
3 minute
1 hour
25 minute

4. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [3]:
import pandas as pd
import re

# Чтение файла
df = pd.read_excel('steps_sample.xlsx')

# Нахождение шаблона в начале строки
pattern = r'^this\.{3},?\s*but'
matches = df['step'].str.contains(pattern, regex=True)

# Вывод количества рецептов и 3 примеров подходящих описаний
print('Количество рецептов:', matches.sum())
print('Примеры подходящих описаний:')
print(df.loc[matches].head(3)['step'])

Количество рецептов: 0
Примеры подходящих описаний:
Series([], Name: step, dtype: object)


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [4]:
import re
import pandas as pd

# читаем файл с шагами рецептов
steps_df = pd.read_excel('steps_sample.xlsx')

# выбираем шаги рецепта с id 72367
recipe_steps = steps_df[steps_df['id'] == 72367]['step']

# убираем пробелы до и после символа дроби в каждом шаге
for step in recipe_steps:
    step = re.sub(r'\s*/\s*', '/', step)
    print(step)

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [1]:
import nltk
import pandas as pd

nltk.download('punkt')

df = pd.read_excel('steps_sample.xlsx')

words = []
for step in df['step']:
    tokens = nltk.word_tokenize(step.lower())
    for token in tokens:
        if token.isalpha():
            words.append(token)

unique_words = set(words)
print(len(unique_words))


14926


[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [14]:
import pandas as pd
import nltk


recipes = pd.read_csv('recipes_sample.csv').dropna(subset=['name', 'description'])


def split_sentences(text):
    return nltk.sent_tokenize(text)


recipes['sentences'] = recipes['description'].apply(split_sentences)

 
recipes['sent_count'] = recipes['sentences'].apply(len)


top_5_longest = recipes.sort_values(by='sent_count', ascending=False).head(5)
print(top_5_longest[['name', 'description', 'sent_count']])

                                                    name  \
18408       my favorite buttercream icing for decorating   
481    alligator claws  avocado fritters  with chipot...   
22566                          rich barley mushroom soup   
16296  little bunny foo foo cake  carrot cake  with c...   
6779                                       chocolate tea   

                                             description  sent_count  
18408  this wonderful icing is used for icing cakes a...          76  
481    a translucent golden-brown crust allows the gr...          27  
22566  this is one of the best soups i've ever made a...          24  
16296  the first time i made this cake i grated a mil...          23  
6779   i wrote this because there are an astounding l...          23  


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.

In [3]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def pos_tagging(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    pos_info = []
    for token, tag in tagged:
        pos_info.append(f'{tag}\t{token}')
    tagged_sentence = ' '.join(pos_info)
    return tagged_sentence
df = pd.read_excel('steps_sample.xlsx')
recipe = df.iloc[0]['step']
print(recipe)
tagged_recipe = pos_tagging(recipe)
print(tagged_recipe)


in 1 / 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes
IN	in CD	1 JJ	/ CD	4 NN	cup NN	butter ,	, JJ	saute NNS	carrots ,	, NN	onion ,	, NN	celery CC	and NN	broccoli VBZ	stems IN	for CD	5 NNS	minutes


[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/datalore/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
import nltk

def display_pos(sentence):
    tokens = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(tokens)
    for tag in pos_tags:
        print("{:<7} ".format(tag[1]), end="")
    print()
    for token in tokens:
        print("{:<7} ".format(token), end="")
    print()
import pandas as pd

df = pd.read_excel("steps_sample.xlsx")
recipe_steps = df[df["id"] == 241106]["step"]
for step in recipe_steps:
    display_pos(step)

TO      VB      DT      JJ      NN      :       VB      DT      JJ      CD      NNS     ,       RB      VBG     DT      NN      CC      VBG     RB      TO      VB      RB      
to      make    the     balsamic marinade :       combine the     first   5       ingredients ,       slowly  adding  the     oil     and     stirring briskly to      combine well    
VBN     RP      
set     aside   
VB      DT      JJ      NN      NN      CC      VB      DT      NN      NN      TO      JJ      
prepare a       hot     charcoal fire    or      preheat a       gas     grill   to      medium-high 
NN      DT      JJ      NN      NN      IN      DT      NN      
place   a       vegetable grilling rack    on      the     grill   
CC      VB      DT      NN      
or      preheat the     broiler 
TO      VB      DT      JJ      NNS     :       VB      DT      JJ      NN      IN      CD      CD      NN      CD      ''      NNS     TO      VB      NNS     
to      make    the     eggplant steaks  :    